This notebook motivates and provides code for a mixture model designed to distinguish patterns found in the ANE from other regions. This model uses continuous weights to account for the additive influence of a number of factors on distributions of typological features.

Of key interest is whether or not a language is in the ANE group. Other factors include family ID and chronological period.

In mixture models, it is a common practice to treat the cluster ID as an unknown variable which must be inferred. Here, since we know which languages are in the ANE and which are not, we can treat this variable as an observed variable. Accordingly, the model becomes similar to multinomial logistic regression, although it assumes many response variables for each data point as opposed to just one.

There are $\mathcal{L}$ languages in the sample. Our observed variables consist of

* $a_{\ell}$, the areal group of language $\ell \in \{\text{ANE},\text{non-ANE}\}$
* $g_{\ell}$, the genetic group of language $\ell \in \{1,...,G\}$ 
* $t_{\ell}$, the date at which language $\ell$ was spoken
* $y_{\ell,d}$ the variant shown by language $\ell$ for feature $d$

Here is the generative process underlying the data:

* For each feature index $d \in \{1,...,D\}$ (e.g., Nominal Duality, etc.)
  * For each feature variant/level $v \in \{1,...,|\text{feature}_d|\}$ (e.g., No Duality)
    * $\beta^0_{d,v} \sim N(0,\sigma)$ (draw a global bias/intercept for the feature variant)
    * $\beta^{\text{time}}_{d,v} \sim N(0,\sigma)$ (draw a parameter representing the influence of chronology on the feature variant, essentially a slope)
    * For each areal group $k \in \{\text{ANE},\text{non-ANE}\}$
      * $w^a_{k,d,v} \sim N(0,\sigma)$ (draw a weight associated with each feature variant in each areal group)
    * For each genetic group $k \in \{1,...,G\}$ (e.g., East Semitic)
      * $w^g_{k,d,v} \sim N(0,\sigma)$ (draw a weight associated with each feature variant in each genetic group)
* For each language $\ell \in \{1,...,\mathcal{L}\}$
  * For each feature index $d \in \{1,...,D\}$
    * For each feature variant/level $v \in \{1,...,|\text{feature}_d|\}$
      * $\psi_{\ell,d,v} = w^a_{a_{\ell},d,v} + w^g_{g_{\ell},d,v} + \beta^0_{d,v} + \beta^{\text{time}}_{d,v}t_{\ell}$
    * $\boldsymbol \phi_{\ell,d} = \text{softmax} (\boldsymbol \psi_{\ell,d})$
    * $y_{\ell,d} \sim \text{Categorical}(\boldsymbol \phi_{\ell,d})$ (draw the observed outcome on the basis of the joint contribution of the language's areal group, genetic group, and date)

Above, the standard deviation $\sigma$ can be fixed or inferred. For simplicity, we can fix it at one. Note that with a small amount of effort, we can use Gaussian distributions with non-diagonal covariance (as opposed to the diagonal covariance assumed here).

Below is a proof of concept using the current data. For lack of a comparison level, we can treat Egyptian as non-ANE and treat all of the variables as categorical (when some perhaps can be recoded as binary), NAs should be removed, etc.

In [1]:
import numpy as np
from collections import defaultdict
import pymc3 as pm
import theano
import theano.tensor as tt
theano.config.gcc.cxxflags = "-fbracket-depth=100000"
import csv

In [2]:
features = []
with open('../features.csv','r') as f:
    for row in csv.reader(f,delimiter=','):
        features.append(row)

In [3]:
ling_data_raw = [l[7:-2] for l in features]
lang_raw = [l[:3] for l in features]
fam_raw = [l[3] for l in features]
area_raw = [l[4] for l in features]
date_raw = [l[-2:] for l in features]

fams = sorted(set([l for l in fam_raw[1:]]))
G = len(fams)

N = len(ling_data_raw)-1

In [4]:
feat_vars = defaultdict(list)
for l in ling_data_raw[1:]:
    for i,f in enumerate(l):
        feat_vars[ling_data_raw[0][i]].append(f)

for k in feat_vars.keys():
    feat_vars[k] = sorted(set(feat_vars[k]))

In [5]:
R = [len(feat_vars[k]) for k in feat_vars.keys()]
X = sum(R)
feat_var_list = [(k,v) for k in feat_vars.keys() for v in feat_vars[k]]

S = [[sum(R[:d]),sum(R[:d])+R[d]] for d in range(len(R))] #the indices for each distribution in the collection

In [6]:
feat_var_id = np.zeros([N,X])
fam_id = np.zeros([N,G])
area_id = np.zeros([N,2])
date_id = np.zeros([N,1])

In [ ]:
for i in range(N):
    for j in range(len(feat_vars.keys())):
        feat_var_id[i,feat_var_list.index((ling_data_raw[0][j],ling_data_raw[i+1][j]))] = 1.
    fam_id[i,fams.index(fam_raw[i+1])] = 1.
    if lang_raw[i+1][0] == 'Egyptian':
        area_id[i,1] = 1.
    else:
        area_id[i,0] = 1.
    date = date_raw[i+1]
    #too lazy to deal with BCE/CE
    date_id[i,0] = np.mean([float(d.split()[0]) for d in date]) + 2000

In [ ]:
def loglik(phi):
    def llik(feat_var_id):
        lliks = tt.log(phi)*feat_var_id
        return(tt.sum(lliks))
    return(llik)

In [ ]:
model = pm.Model()
with model:
    beta_0 = tt.stack([pm.Normal('beta_0_{}'.format(x),0,1) for x in range(X)])
    beta_time = tt.stack([[pm.Normal('beta_time_{}'.format(x),0,1) for x in range(X)]])
    w_a = tt.stack([[pm.Normal('w_a_{}_{}'.format(k,x),0,1) for x in range(X)] for k in range(2)]) #areal group prior
    w_g = tt.stack([[pm.Normal('w_g_{}_{}'.format(k,x),0,1) for x in range(X)] for k in range(G)]) #genetic group prior
    psi = beta_0 + tt.dot(date_id,beta_time) + tt.dot(area_id,w_a) + tt.dot(fam_id,w_g)
    phi = tt.concatenate([tt.nnet.softmax(psi[:,S[d][0]:S[d][1]]) for d in range(len(R))],-1)
    target = pm.DensityDist('target',loglik(phi=phi),observed=dict(feat_var_id=feat_var_id))
    inference = pm.ADVI()
    inference.fit(100000, obj_optimizer=pm.adam(learning_rate=.01,beta1=.8),callbacks=[pm.callbacks.CheckParametersConvergence()])
    trace = inference.approx.sample()
    posterior = {k:trace[k] for k in trace.varnames if not k.endswith('__')}
    posterior['ELBO'] = inference.hist
